In [19]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from tensorflow import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16


In [2]:
SIZE = 64
train_images = []
train_labels = []

for directory_path in glob.glob("train/*"):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path,"*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(SIZE,SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)


malignant
benign


In [3]:
test_images = []
test_labels = []

for directory_path in glob.glob("test/*"):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path,"*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(SIZE,SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(label)

malignant
benign


In [4]:
print(np.array(test_images).shape)

(3000, 64, 64, 3)


In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

# trocar o nome das variáveis
train_images = np.array(train_images)
test_images = np.array(test_images)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded


# Normalizar
x_train = x_train/255.0
x_test = x_test/255.0

from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [16]:
# Definição dos modelo
#Load model wothout classifier/fully connected layers
VGG_model = VGG16(include_top=False, weights='imagenet', input_shape=(SIZE, SIZE, 3))

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()  #Trainable parameters will be 0



Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [18]:
out_vgg = VGG_model.predict(x_train)

print(out_vgg.shape)
print(out_vgg[0])


219/219 [==============================] - 98s 448ms/step
(7000, 2, 2, 512)
[[[0.01645227 0.         0.33360118 ... 0.         1.1429665  0.        ]
  [0.34460384 0.         0.403619   ... 0.         0.78148204 0.        ]]

 [[0.         0.         0.         ... 0.         1.1382532  0.        ]
  [0.         0.         0.         ... 0.         0.91165644 0.        ]]]


In [27]:
DNN_model = Sequential()
DNN_model.add(Dense(2048,activation="relu",input_shape=(2048, )))
DNN_model.add(Dense(4096,activation="relu"))
DNN_model.add(Dense(2,activation="sigmoid"))

DNN_model.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

print(DNN_model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_7 (Dense)             (None, 4096)              8392704   
                                                                 
 dense_8 (Dense)             (None, 2)                 8194      
                                                                 
Total params: 12597250 (48.05 MB)
Trainable params: 12597250 (48.05 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [29]:
out_vgg = np.array([x.flatten() for x in out_vgg])

DNN_model.fit(out_vgg,y_train_one_hot, epochs=10, verbose=2)



Epoch 1/10
219/219 - 32s - loss: 0.2088 - accuracy: 0.9159 - 32s/epoch - 147ms/step
Epoch 2/10
219/219 - 32s - loss: 0.1763 - accuracy: 0.9311 - 32s/epoch - 145ms/step
Epoch 3/10
219/219 - 32s - loss: 0.1630 - accuracy: 0.9359 - 32s/epoch - 147ms/step
Epoch 4/10
219/219 - 32s - loss: 0.1483 - accuracy: 0.9420 - 32s/epoch - 146ms/step
Epoch 5/10
219/219 - 32s - loss: 0.1276 - accuracy: 0.9491 - 32s/epoch - 146ms/step
Epoch 6/10
219/219 - 32s - loss: 0.1215 - accuracy: 0.9513 - 32s/epoch - 147ms/step
Epoch 7/10
219/219 - 32s - loss: 0.1030 - accuracy: 0.9619 - 32s/epoch - 146ms/step
Epoch 8/10
219/219 - 32s - loss: 0.1011 - accuracy: 0.9584 - 32s/epoch - 147ms/step
Epoch 9/10
219/219 - 32s - loss: 0.0894 - accuracy: 0.9670 - 32s/epoch - 147ms/step
Epoch 10/10
219/219 - 34s - loss: 0.0796 - accuracy: 0.9701 - 34s/epoch - 156ms/step


In [31]:
predict_imgs = VGG_model.predict(x_test)

predict_imgs = np.array([x.flatten() for x in predict_imgs])

preditc_out = DNN_model.predict(predict_imgs)




94/94 [==============================] - 1s 11ms/step


In [36]:
for i in range(0,len(preditc_out)):
    preditc_out[i][0] = 1 if preditc_out[i][0] > 0.5 else 0
    preditc_out[i][1] = 1 if preditc_out[i][1] > 0.5 else 0

In [43]:
conf_mat = np.array([[0,0],[0,0]])

for i in range(0,len(preditc_out)):
    if preditc_out[i][0] == 1:
        if y_test_one_hot[i][0]:
            conf_mat[0][0] += 1
        else:
            conf_mat[0][1] += 1
    else:
        if y_test_one_hot[i][0]:
            conf_mat[1][0] += 1
        else:
            conf_mat[1][1] += 1

print(conf_mat)


[[1361  302]
 [ 139 1198]]
